In [ ]:
import time
import random
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Lista de proxies SOCKS5 disponibles
ips = [
    "socks5://103.35.108.206:1080",
    "socks5://199.188.93.214:1080",
    "socks5://152.32.186.145:1080",
    "socks5://119.8.111.196:1080",
    "socks5://213.5.17.84:1080"
]

# Ruta del archivo Excel donde se guardarán los resultados
excel_path = r"C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\COSCU.xlsx"

# Función para configurar el navegador con una IP SOCKS5 específica
def get_driver_with_ip(ip):
    chrome_options = Options()
    chrome_options.add_argument("--disable-notifications")
    chrome_options.add_argument(f"--proxy-server={ip}")
    chrome_options.add_argument("--proxy-bypass-list=*")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

# Función para realizar el web scraping de una página de productos
def scrape_page(url):
    # Seleccionar un proxy aleatorio
    ip = random.choice(ips)
    
    # Configurar el navegador con el proxy
    driver = get_driver_with_ip(ip)
    
    driver.get(url)
    
    # Esperar a que la página cargue completamente
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "nikecl-search-result-3-x-galleryItem")))
    
    # Lista para almacenar los resultados
    products = []
    
    # Extraer productos
    product_elements = driver.find_elements(By.CLASS_NAME, "nikecl-search-result-3-x-galleryItem")
    
    for product in product_elements:
        try:
            # Nombre del producto
            name = product.find_element(By.CLASS_NAME, "vtex-product-summary-2-x-productBrand").text
        except:
            name = ""
        
        # Precio original (si existe)
        try:
            original_price = product.find_element(By.CLASS_NAME, "vtex-product-price-1-x-currencyContainer").text
        except:
            original_price = ""
        
        # Precio final (si existe)
        try:
            final_price = product.find_element(By.CLASS_NAME, "vtex-product-price-1-x-sellingPriceValue").text
        except:
            final_price = ""
        
        # URL de la imagen
        try:
            image_url = product.find_element(By.TAG_NAME, "img").get_attribute("src")
        except:
            image_url = ""
        
        # URL del producto
        try:
            product_url = product.find_element(By.TAG_NAME, "a").get_attribute("href")
        except:
            product_url = ""
        
        # Guardar los datos en la lista
        products.append({
            'Nombre': name,
            'Precio Original': original_price,
            'Precio Final': final_price,
            'Imagen': image_url,
            'URL': product_url
        })
    
    # Cerrar el navegador
    driver.quit()
    
    return products

# Función para guardar los resultados en el archivo Excel
def save_to_excel(products):
    df = pd.DataFrame(products)
    df.to_excel(excel_path, index=False)
    print(f"Datos guardados en {excel_path}")

# Función para navegar por todas las páginas de productos
def scrape_nike_all_pages():
    all_products = []
    page = 1
    
    while page <= 16:  # Terminar después de la página 16
        # Construir la URL para la página actual
        url = f"https://www.nike.cl/ninos/zapatillas?page={page}"
        
        # Scrapeo de la página actual
        products = scrape_page(url)
        
        # Si no se encontraron productos, hemos llegado al final
        if not products:
            break
        
        # Agregar los productos encontrados a la lista total
        all_products.extend(products)
        
        # Aumentar el número de página
        page += 1
        print(f"Página {page - 1} procesada.")
        time.sleep(random.uniform(2, 5))  # Esperar un poco antes de cargar la siguiente página
    
    # Guardar todos los productos en el archivo Excel
    save_to_excel(all_products)

# Ejecutar el scraping
scrape_nike_all_pages()


Página 1 procesada.
Página 2 procesada.
Página 3 procesada.
Página 4 procesada.
Página 5 procesada.
Página 6 procesada.
Página 7 procesada.
Página 8 procesada.
Página 9 procesada.
Página 10 procesada.
Página 11 procesada.
